In [ ]:
from ultralytics import YOLO
import ultralytics
import os

# Load a pretrained YOLOv8n model
model = YOLO('/Data4/student_zhihan_data/source_code/yolo/ultralytics/runs/detect/GC10-DET_brightness_0 detect by yolov8n with dropout(p=0.1)/weights/best.pt')

# Define path to directory containing images and videos for inference
source = '/Data4/student_zhihan_data/data/GC10-DET/test/images'


# Run inference on the source
results = model([os.path.join(source, i) for i in os.listdir(source)])# generator of Results objects


0: 640x640 1 crescent_gap, 1 welding_line, 1: 640x640 2 silk_spots, 2 water_spots, 2: 640x640 1 oil_spot, 1 rolled_pit, 3: 640x640 1 water_spot, 4: 640x640 1 crescent_gap, 1 silk_spot, 1 welding_line, 5: 640x640 1 silk_spot, 6: 640x640 2 oil_spots, 3 punching_holes, 1 welding_line, 7: 640x640 (no detections), 8: 640x640 1 waist_folding, 9: 640x640 1 waist_folding, 10: 640x640 1 crescent_gap, 1 welding_line, 11: 640x640 1 crescent_gap, 12: 640x640 1 oil_spot, 13: 640x640 1 silk_spot, 14: 640x640 1 silk_spot, 15: 640x640 2 creases, 16: 640x640 1 oil_spot, 17: 640x640 1 oil_spot, 1 water_spot, 18: 640x640 (no detections), 19: 640x640 1 crescent_gap, 1 welding_line, 20: 640x640 2 crescent_gaps, 2 welding_lines, 21: 640x640 (no detections), 22: 640x640 1 punching_hole, 23: 640x640 1 welding_line, 24: 640x640 1 punching_hole, 25: 640x640 2 silk_spots, 26: 640x640 1 punching_hole, 1 welding_line, 27: 640x640 1 water_spot, 28: 640x640 2 water_spots, 29: 640x640 (no detections), 30: 640x640 1 

In [42]:
from ultralytics.engine.validator import BaseValidator
from ultralytics.utils.metrics import box_iou, Metric, DetMetrics
import numpy as np
import torch

# load txt label


validator = BaseValidator()
validator.iouv = torch.arange(0.5, 1, 0.05)

for result in results:
    metric = Metric()
    label_path = result.path.replace('images', 'labels')[:-4] + '.txt'
    label = torch.from_numpy(np.loadtxt(label_path))
    # turn label into (x1, y1, x2, y2) format
    
    if label.dim() > 1:
        x1 = label[:, 1] - label[:, 3] / 2
        y1 = label[:, 2] - label[:, 4] / 2
        x2 = label[:, 1] + label[:, 3] / 2
        y2 = label[:, 2] + label[:, 4] / 2
        label = torch.stack((label[:, 0], x1, y1, x2, y2), 1)
        iou = box_iou(label[:, 1:].to("cuda:0"), result.boxes.xyxyn.to("cuda:0"))
        tp = validator.match_predictions(result.boxes.data[:,-1], label[:, 0].to("cuda:0"), iou)
    else:
        x1 = label[1] - label[3] / 2
        y1 = label[2] - label[4] / 2
        x2 = label[1] + label[3] / 2
        y2 = label[2] + label[4] / 2
        label = torch.tensor([label[0], x1, y1, x2, y2])
        iou = box_iou(label[1:].to("cuda:0").reshape(1, -1), result.boxes.xyxyn.to("cuda:0"))
        tp = validator.match_predictions(result.boxes.data[:,-1], label[0].to("cuda:0").unsqueeze(0), iou)
    
    # update metric
    
    
    if torch.sum(tp) > 0:
        print(tp)
      

tensor([[ True,  True,  True,  True,  True,  True, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True, False, False]], device='cuda:0')
tensor([[ True,  True,  True,  True, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False,  True, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False]], device='cuda:0')
tensor([[ True,  True,  True,  True,  True,  True,  True, False, False, False]], device='cuda:0')
tensor([[ True,  True,  True,  True,  True, False, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False]], device='cuda:0')
tensor([[ True,  True,  True,  True, False, False, False, False, False, False]], device='cuda:0')
tensor([[ True,  True,  True, 

In [2]:
# Process results generator
for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs


KeyboardInterrupt: 